In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import model_analyzer as ma
import ml_pipe as ml
import combined_features_rachel as cf
import epa_pipeline as ep
import warnings
warnings.filterwarnings('ignore')

In [2]:
facs, evals, vios, snc = ep.temp_holdout_prep('RCRA_FACILITIES.csv', 'RCRA_EVALUATIONS.csv', 'RCRA_VIOLATIONS.csv',
                                             'RCRA_VIOSNC_HISTORY.csv', '2011-01-01')

evals begin:  2011-01-01 00:00:00
evals end:  2019-09-19 00:00:00
datetime64[ns]
vios begin:  2011-01-03 00:00:00
vios end:  2019-09-27 00:00:00
SNC begin:  2011-02-01 00:00:00
SNC end:  2019-04-01 00:00:00


In [3]:
trains, tests, train_ends, test_ends = ep.temporal_split(evals, vios, snc)

FINAL INFO FROM snc_info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041254 entries, 0 to 1041253
Data columns (total 25 columns):
ID_NUMBER                    1041254 non-null object
FACILITY_NAME                1041248 non-null object
ACTIVITY_LOCATION            1041254 non-null object
FULL_ENFORCEMENT             1041254 non-null object
HREPORT_UNIVERSE_RECORD      1041157 non-null object
STREET_ADDRESS               1041118 non-null object
CITY_NAME                    1041152 non-null object
STATE_CODE                   1041253 non-null object
ZIP_CODE                     1041053 non-null object
LATITUDE83                   452581 non-null float64
LONGITUDE83                  452581 non-null float64
FED_WASTE_GENERATOR          1032663 non-null object
TRANSPORTER                  1040960 non-null object
ACTIVE_SITE                  1041254 non-null object
OPERATING_TSDF               1041254 non-null object
EvalCount                    1041254 non-null float64
Sum_Violation

In [ ]:
feature_lst = ['ID_NUMBER','population', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 
'B02001_006E', 'B02001_007E', 'B02001_008E', 'B02001_009E', 'B02001_010E', 
'B11016_002E', 'B11016_003E', 'B11016_004E', 'B11016_005E', 'B11016_006E', 
'B11016_007E', 'B11016_008E', 'B17026_002E', 'B17026_003E', 'B17026_004E', 
'B17026_005E', 'B17026_006E', 'B17026_007E', 'B17026_008E', 'B17026_009E', 
'B17026_010E', 'B17026_011E', 'B17026_012E', 'B17026_013E', 'B19001_002E', 
'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E', 'B19001_007E',
'B19001_008E', 'B19001_009E', 'B19001_010E', 'B19001_011E', 'B19001_012E', 
'B19001_013E', 'B19001_014E', 'B19001_015E', 'B19001_016E', 'B19001_017E', 
'median income', 'C17002_002E', 'C17002_003E', 'C17002_004E', 'C17002_005E',
'C17002_006E', 'C17002_007E', 'C17002_008E','white alone', 'black alone', 
'native alone', 'asian alone', 'pacific alone', 'other alone', 'two or more', 
'two or more some other', 'less10k', '10kto15k', '15kto20k', '20kto25k', '25kto30k',
'30kto35k', '35kto40k', '40kto45k', '1 person', '2 person', '3 person', '4 person', 
'5 person', '6 person', '7plusperson', 'under_p5_x', 'p5top74', 'p75top99', '1to1p24_x',
'1p25to1p49_x', '1p50to1p74', '1p75to1p84', '1p85to1p99_x', 'under_p5_y', 'p5top99', 
'1to1p24_y', '1p25to1p49_y', '1p50to1p84', '1p85to1p99_y', '2andver']
feature_reduced = ['ID_NUMBER','population','black alone', 'median income','under_p5_x']

In [ ]:
list_of_trainx = [x[feature_lst] for x in trains]
list_of_trainy = [x['HadViolation'] for x in trains]
list_of_testx = [x[feature_lst] for x in tests]
list_of_testy = [x['HadViolation'] for x in tests]

In [ ]:
clfs, grid = ml.define_clfs_params('dtandlr_explore')

In [ ]:
thresholds = [.8]

In [ ]:
feats = ml.select_features(features)

In [ ]:
predictions, models, stats = ml.model_analyzer_over_time(clfs, grid, 'show', thresholds, train_xs, train_ys, test_xs, test_ys, feats)